In [1]:
from moccasin import setup_notebook

setup_notebook()

## Get active network for moccasin

this is what we gonna do:
1. Add deposit to our wallet
2. Deposit the balance to aave
3. withdraw the fund
4. rebalance the portfolio

Now, let's start with active network

In [2]:
from moccasin.config import get_active_network

active_network = get_active_network()
print(active_network)
eth_usd = active_network.manifest_named("eth_usd_price_feed")
print(eth_usd)

Network(name='arbitrum-fork', url='https://arbitrum-mainnet.infura.io/v3/fc95abf6201d4f99bd29909d2aa63d68', chain_id=42161, is_fork=True, block_identifier='safe', is_zksync=False, default_account_name=None, unsafe_password_file=None, save_abi_path='abis', explorer_uri='https://api.arbiscan.io', explorer_api_key='WA36QTQE54UDXCXQR3KEEGGKNEJXKI6DII', explorer_type=None, named_contracts={'weth': NamedContract(contract_name='weth', force_deploy=None, abi='abis/weth.json', abi_from_explorer=None, deployer_script=None, address='0x82aF49447D8a07e3bd95BD0d56f35241523fBab1', deployer=None, recently_deployed_contract=None), 'usdc': NamedContract(contract_name='usdc', force_deploy=None, abi='abis/usdc.json', abi_from_explorer=None, deployer_script=None, address='0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8', deployer=None, recently_deployed_contract=None), 'uniswap_swap_router': NamedContract(contract_name='uniswap_swap_router', force_deploy=None, abi='abis/uniswap_swap_router.json', abi_from_explo

Now after we connect to the network, we need to deposit some eth for the gas fees, we can use titanoboa set_balance on forked network

# Deposit to wallet

In [3]:
from script._setup_script import _add_eth_balance, _add_token_balance, setup_script
# from moccasin import get_config

# active_config = get_config()
usdc, weth, a_usdc, a_weth = setup_script()
    


Active network: arbitrum-fork


USDC contract address: 0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8
WETH contract address: 0x82aF49447D8a07e3bd95BD0d56f35241523fBab1
Setting up local network with initial balances...
Added 1000000000000000000000 wei to 0xd13f0Bd22AFF8176761AEFBfC052a7490bDe268E
wallet address: 0xd13f0Bd22AFF8176761AEFBfC052a7490bDe268E
Adding 10000000 USDC and 13500000000000000000 WETH to 0xd13f0Bd22AFF8176761AEFBfC052a7490bDe268E


BoaError: 
========================================================================
[E] [7493] usdc.sendValue(to = "0x1eED63EfBA5f81D95bfe37d82C8E736b974F477b", value = 10000000) <0x>
    [E] [249] Unknown contract 0x1eFB3f88Bc88f03FD1804A5C53b7141bbEf5dED8.0x24a084df
========================================================================

  (<usdc interface at 0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8>.sendValue(address,uint256) -> ['bool'])

   <Unknown contract 0x1efb3f88bc88f03fd1804a5c53b7141bbef5ded8>

  (<usdc interface at 0xFF970A61A04b1cA14834A43f5dE4533eBDDB5CC8>.sendValue(address,uint256) -> ['bool'])

# Deposit to AAVE

After we deposited to our wallet, we want to deposit the money to Aave, we use supply function here

In [ ]:
from script.deposit import run_deposit_script

run_deposit_script(usdc, weth)

After we deposit to Aave, we want to rebalance the balance

# Rebalance

In [ ]:
from script.rebalance import rebalance

rebalance(usdc, weth, a_usdc, a_weth)

starting_usdc_balance = usdc.balanceOf(boa.env.eoa)
starting_weth_balance = weth.balanceOf(boa.env.eoa)

print(f"Wallet after deposit USDC balance: {starting_usdc_balance / 1e6} USDC")
print(f"Wallet after deposit WETH balance: {starting_weth_balance / 1e18} WETH")

Now, we will withdraw the token and deposit again

In [ ]:
import boa

starting_usdc_balance = usdc.balanceOf(boa.env.eoa)
starting_weth_balance = weth.balanceOf(boa.env.eoa)

print(f"Wallet after deposit USDC balance: {starting_usdc_balance / 1e6} USDC")
print(f"Wallet after deposit WETH balance: {starting_weth_balance / 1e18} WETH")

run_deposit_script(usdc, weth)

a_usdc_balance = a_usdc.balanceOf(boa.env.eoa)
a_weth_balance = a_weth.balanceOf(boa.env.eoa)

print(f"Current Aave USDC balance: {a_usdc_balance / 1e6} USDC")
print(f"Current Aave WETH balance: {a_weth_balance / 1e18} WETH")

# print(f"Wallet after rebalance USDC balance: {starting_usdc_balance / 1e6} USDC")
# print(f"Wallet after rebalance WETH balance: {starting_weth_balance / 1e18} WETH")

Let's check the price and total value

In [ ]:
from script.rebalance import get_price

a_usdc_balance_normalized = a_usdc_balance / 1e6
a_weth_balance_normalized = a_weth_balance / 1e18

usdc_value = a_usdc_balance_normalized * get_price("usdc_usd_price_feed")
weth_value = a_weth_balance_normalized * get_price("eth_usd_price_feed")

print(f"Current aUSDC value: ${usdc_value:.2f}")
print(f"Current aWETH value: ${weth_value:.2f}")

total_value = usdc_value + weth_value
print(f"Current percent allocation of aUSDC: {usdc_value / total_value * 100:.2f}%")
print(f"Current percent allocation of aWETH: {weth_value / total_value * 100:.2f}%")